In [5]:
import torch
from torch import nn, device
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np

In [6]:
data=np.load('../data/mnist.npz')
x_train=torch.tensor(data['x_train'],dtype=torch.float).reshape(-1,1,28,28)
y_train=torch.tensor(data['y_train'],dtype=torch.int64)
x_test=torch.tensor(data['x_test'],dtype=torch.float).reshape(-1,1,28,28)
y_test=torch.tensor(data['y_test'],dtype=torch.int64)


In [7]:

net=nn.Sequential(nn.Conv2d(1,6,kernel_size=5,stride=1,padding=2),nn.Sigmoid(),nn.AvgPool2d(kernel_size=2,stride=2)
                  ,nn.Conv2d(6,16,kernel_size=5),nn.Sigmoid(),nn.MaxPool2d(kernel_size=2,stride=2),nn.Flatten(),nn.Linear(16*5*5,120)
                  ,nn.Linear(120,84),nn.Sigmoid(),nn.Linear(84,10))


In [8]:
X = torch.rand(size=(1, 1, 28, 28), dtype=torch.float32)
for layer in net:
     X = layer(X)
     print(layer.__class__.__name__,'output shape: \t',X.shape)

Conv2d output shape: 	 torch.Size([1, 6, 28, 28])
Sigmoid output shape: 	 torch.Size([1, 6, 28, 28])
AvgPool2d output shape: 	 torch.Size([1, 6, 14, 14])
Conv2d output shape: 	 torch.Size([1, 16, 10, 10])
Sigmoid output shape: 	 torch.Size([1, 16, 10, 10])
MaxPool2d output shape: 	 torch.Size([1, 16, 5, 5])
Flatten output shape: 	 torch.Size([1, 400])
Linear output shape: 	 torch.Size([1, 120])
Linear output shape: 	 torch.Size([1, 84])
Sigmoid output shape: 	 torch.Size([1, 84])
Linear output shape: 	 torch.Size([1, 10])


In [42]:
from d2l import torch as d2l
lr=0.1
batch_size=256
num_epoch=100

In [10]:
class my_datasets(torch.utils.data.Dataset):
    def __init__(self,data,transform=None):
        self.data=data
        self.transform=transform
        self.x=data[0]
        print(self.data[1].shape)
    def __len__(self):
        return len(self.x)
    def __getitem__(self, idx):
        sample=self.data[0][idx],self.data[1][idx]
        
        if self.transform:
            sample=self.transform(sample[0]),self.transform(sample[1])
        return sample
        

In [35]:
from torchvision import transforms
from torch.utils import data
train_iter=data.DataLoader(my_datasets((x_train,y_train)),batch_size=batch_size,shuffle=True)
test_iter=data.DataLoader(my_datasets((x_test,y_test)),batch_size=batch_size,shuffle=False)

torch.Size([60000])
torch.Size([10000])


In [12]:
import torchvision.datasets
trans=transforms.ToTensor()#使plt类型变成float32
mnist_train=torchvision.datasets.FashionMNIST(root='E:\pytorch\data', train=True, transform=trans, download=True)
mnist_test=torchvision.datasets.FashionMNIST(root='E:\pytorch\data', train=False, transform=trans)
mnist_train_iter=torch.utils.data.DataLoader(mnist_train,batch_size=batch_size,shuffle=True)
mnist_test_iter=torch.utils.data.DataLoader(mnist_test,batch_size=batch_size,shuffle=False)

In [36]:
next(iter(mnist_train_iter))[1]

tensor([7, 8, 0, 8, 7, 2, 9, 7, 3, 9, 2, 3, 2, 5, 1, 5, 2, 9, 3, 1, 4, 4, 1, 1,
        0, 5, 2, 5, 5, 0, 2, 5, 4, 6, 0, 6, 6, 3, 3, 2, 7, 7, 9, 3, 4, 4, 6, 3,
        6, 7, 9, 6, 8, 4, 9, 1, 5, 5, 5, 2, 8, 6, 8, 7, 2, 2, 1, 8, 8, 4, 2, 0,
        3, 3, 6, 6, 8, 3, 5, 2, 7, 0, 1, 2, 5, 0, 4, 2, 3, 7, 6, 8, 5, 1, 0, 0,
        8, 5, 4, 9, 1, 9, 0, 3, 0, 4, 7, 4, 8, 7, 3, 9, 1, 5, 0, 5, 2, 5, 0, 1,
        9, 6, 5, 6, 5, 1, 7, 5, 4, 5, 5, 8, 1, 3, 5, 5, 1, 2, 4, 3, 1, 7, 6, 8,
        7, 3, 2, 9, 4, 5, 4, 6, 4, 0, 8, 8, 9, 6, 1, 0, 7, 0, 9, 7, 6, 4, 1, 6,
        9, 4, 0, 0, 1, 9, 2, 9, 2, 9, 6, 3, 1, 4, 4, 0, 9, 0, 5, 6, 9, 4, 5, 0,
        7, 8, 5, 2, 1, 3, 7, 4, 1, 1, 3, 5, 7, 3, 5, 5, 9, 6, 0, 5, 0, 6, 0, 5,
        7, 8, 9, 0, 3, 5, 9, 0, 5, 8, 1, 0, 0, 5, 8, 3, 6, 3, 8, 6, 8, 5, 1, 9,
        4, 0, 0, 2, 1, 4, 4, 6, 5, 6, 5, 6, 7, 5, 7, 2])

In [37]:
def evaluate_accuracy_gpu(net, data_iter, device=None): #@save
    """使用GPU计算模型在数据集上的精度"""
    if isinstance(net, nn.Module):
        net.eval() # 设置为评估模式
    if not device:
        device = next(iter(net.parameters())).device
# 正确预测的数量，总预测的数量
    metric = d2l.Accumulator(2)
    with torch.no_grad():
        for X, y in data_iter:
            if isinstance(X, list):
    # BERT微调所需的（之后将介绍）
                X = [x.to(device) for x in X]
            else:
                X = X.to(device)
                y = y.to(device)
            metric.add(d2l.accuracy(net(X), y), y.numel())
    return metric[0] / metric[1]

In [48]:
def train_ch6(net, train_iter, test_iter, num_epochs, lr, device):
    """Train a model with a GPU (defined in Chapter 6).

    Defined in :numref:`sec_utils`"""
    def init_weights(m):
        if type(m) == nn.Linear or type(m) == nn.Conv2d:
            nn.init.xavier_uniform_(m.weight)
    net.apply(init_weights)
    print('training on', device)
    net.to(device)
    optimizer = torch.optim.SGD(net.parameters(), lr=lr)
    loss = nn.CrossEntropyLoss()
    animator = d2l.Animator(xlabel='epoch', xlim=[1, num_epochs],
                            legend=['train loss', 'train acc', 'test acc'])
    timer, num_batches = d2l.Timer(), len(train_iter)
    for epoch in range(num_epochs):
        # Sum of training loss, sum of training accuracy, no. of examples
        metric = d2l.Accumulator(3)
        net.train()
        for i, (X, y) in enumerate(train_iter):
            timer.start()
            optimizer.zero_grad()
            X, y = X.to(device), y.to(device)
            y_hat = net(X)
            l = loss(y_hat, y)
            
            
            l.backward()
            optimizer.step()
            with torch.no_grad():
                metric.add(l * X.shape[0], d2l.accuracy(y_hat, y), X.shape[0])
              
            timer.stop()
            train_l = metric[0] / metric[2]
            train_acc = metric[1] / metric[2]
            if (i + 1) % (num_batches // 5) == 0 or i == num_batches - 1:
                animator.add(epoch + (i + 1) / num_batches,
                             (train_l, train_acc, None))
        test_acc = evaluate_accuracy_gpu(net, test_iter)
        animator.add(epoch + 1, (None, None, test_acc))
    print(f'loss {train_l:.3f}, train acc {train_acc:.3f}, '
          f'test acc {test_acc:.3f}')
    print(f'{metric[2] * num_epochs / timer.sum():.1f} examples/sec '
          f'on {str(device)}')
    


In [ ]:

train_ch6(net, train_iter, test_iter,num_epoch,lr,d2l.try_gpu())


In [ ]:
torch.save(net.state_dict(), './model_1.pth')

In [ ]:
x=torch.rand(size=(1,1, 28, 28), dtype=torch.float32)

In [ ]:
model=net=nn.Sequential(nn.Conv2d(1,6,kernel_size=5,stride=1,padding=2),nn.Sigmoid(),nn.AvgPool2d(kernel_size=2,stride=2)
                  ,nn.Conv2d(6,16,kernel_size=5),nn.Sigmoid(),nn.MaxPool2d(kernel_size=2,stride=2),nn.Flatten(),nn.Linear(16*5*5,120)
                  ,nn.Linear(120,84),nn.Sigmoid(),nn.Linear(84,10))
model.load_state_dict(torch.load('./model_1.pth'))
model.eval()
model(x),net(x)